In [1]:
import os
import numpy as np
import cv2
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import moments_hu
from tqdm import tqdm

# Function to extract color features (mean and std of RGB channels)
def extract_color_features(image):
    mean = np.mean(image, axis=(0, 1))  # Mean per channel
    std = np.std(image, axis=(0, 1))    # Std deviation per channel
    return np.concatenate([mean, std])

# Function to extract texture features using GLCM
def extract_texture_features(image):
    gray_image = rgb2gray(image)  # Convert to grayscale
    gray_image = (gray_image * 255).astype(np.uint8)  # Scale to 8-bit
    
    glcm = graycomatrix(gray_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    return np.array([contrast, dissimilarity, homogeneity, energy, correlation])

# Function to extract shape features using Hu Moments
def extract_shape_features(image):
    gray_image = rgb2gray(image)  # Convert to grayscale
    gray_image = (gray_image * 255).astype(np.uint8)  # Convert to 8-bit

    moments = moments_hu(gray_image)
    log_moments = -np.sign(moments) * np.log10(np.abs(moments))  # Log scale for better representation
    return log_moments

# Paths for input dataset and output files
input_dir = "Resized_IMG_CLASSES_2"
features_output_file = "ovs_manual_features.npy"
labels_output_file = "ovs_manual_labels.npy"

# Prepare data storage
features_list = []
labels_list = []

# Process images
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue  # Skip non-folder items

    print(f"Extracting features from class: {class_name}")

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            continue  # Skip unreadable images

        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Extract features
        color_features = extract_color_features(image)
        texture_features = extract_texture_features(image)
        shape_features = extract_shape_features(image)

        # Combine all features
        features = np.concatenate([color_features, texture_features, shape_features])
        features_list.append(features)
        labels_list.append(class_name)  # Store label

# Convert lists to NumPy arrays
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Save as .npy files
np.save(features_output_file, features_array)
np.save(labels_output_file, labels_array)

print(f"Feature extraction complete! Features saved in {features_output_file} and labels in {labels_output_file}")


Extracting features from class: 1. Eczema


Processing 1. Eczema:   3%|▎         | 49/1500 [00:00<00:12, 120.70it/s]C:\Users\RYZEN\AppData\Local\Temp\ipykernel_12356\2665905623.py:36: RuntimeWarning: divide by zero encountered in log10
  log_moments = -np.sign(moments) * np.log10(np.abs(moments))  # Log scale for better representation
C:\Users\RYZEN\AppData\Local\Temp\ipykernel_12356\2665905623.py:36: RuntimeWarning: invalid value encountered in multiply
  log_moments = -np.sign(moments) * np.log10(np.abs(moments))  # Log scale for better representation
Processing 1. Eczema: 100%|██████████| 1500/1500 [00:12<00:00, 118.57it/s]


Extracting features from class: 2. Melanoma


Processing 2. Melanoma: 100%|██████████| 1000/1000 [00:08<00:00, 113.00it/s]


Extracting features from class: 3. Atopic Dermatitis


...

Extracting features from class: 6. Fungal Infections


Processing 6. Fungal Infections: 100%|██████████| 1500/1500 [00:12<00:00, 122.05it/s]


Extracting features from class: 7. Viral Infections


Processing 7. Viral Infections: 100%|██████████| 1500/1500 [00:12<00:00, 124.16it/s]

Feature extraction complete! Features saved in ovs_manual_features.npy and labels in ovs_manual_labels.npy


In [3]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "oversampled_normalized_images/densenet"
output_feature_path = "oversampled_featuresets/ovs_densenet_features.npy"
output_label_path = "oversampled_featuresets/ovs_densenet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class DenseNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = DenseNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load DenseNet121 and remove classifier
densenet = models.densenet121(pretrained=True)
densenet.classifier = torch.nn.Identity()  # Remove final classification layer
densenet = densenet.to(device)
densenet.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting DenseNet121 features"):
        images = images.to(device)
        outputs = densenet(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("DenseNet121 features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting DenseNet121 features: 100%|██████████| 274/274 [07:21<00:00,  1.61s/it]

DenseNet121 features saved to: oversampled_featuresets/ovs_densenet_features.npy
Labels saved to: oversampled_featuresets/ovs_densenet_labels.npy


In [4]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "oversampled_normalized_images/mobilenetv2"
output_feature_path = "oversampled_featuresets/ovs_mobilenet_features.npy"
output_label_path = "oversampled_featuresets/ovs_mobilenet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class MobileNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = MobileNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load MobileNetV2 and remove classifier
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier = torch.nn.Identity()  # Remove classification layer
mobilenet = mobilenet.to(device)
mobilenet.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting MobileNetV2 features"):
        images = images.to(device)
        outputs = mobilenet(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("MobileNetV2 features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting MobileNetV2 features: 100%|██████████| 274/274 [03:12<00:00,  1.42it/s]

MobileNetV2 features saved to: oversampled_featuresets/ovs_mobilenet_features.npy
Labels saved to: oversampled_featuresets/ovs_mobilenet_labels.npy


In [5]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Paths
data_dir = "oversampled_normalized_images/resnet"
output_feature_path = "oversampled_featuresets/ovs_resnet_features.npy"
output_label_path = "oversampled_featuresets/ovs_resnet_labels.npy"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class ResNetImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = sorted(os.listdir(root_dir))
        for label_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label_index)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Transformation
transform = transforms.ToTensor()

# Dataset and DataLoader
dataset = ResNetImageDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load ResNet50 and remove the final FC layer
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity()  # Remove classification layer
resnet = resnet.to(device)
resnet.eval()

# Extract features
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Extracting ResNet50 features"):
        images = images.to(device)
        outputs = resnet(images)
        features_list.append(outputs.cpu().numpy())
        labels_list.extend(labels.numpy())

# Save features and labels
features_array = np.vstack(features_list)
labels_array = np.array(labels_list)

np.save(output_feature_path, features_array)
np.save(output_label_path, labels_array)

print("ResNet50 features saved to:", output_feature_path)
print("Labels saved to:", output_label_path)


D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting ResNet50 features: 100%|██████████| 274/274 [07:24<00:00,  1.62s/it]

ResNet50 features saved to: oversampled_featuresets/ovs_resnet_features.npy
Labels saved to: oversampled_featuresets/ovs_resnet_labels.npy
